The given tutorial gives a brief introduction on how time series dataset can be with Convolutional Neural Network architechture for developing a simple CNN classifier.
(API Reference: https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.classification.deep_learning.CNNClassifier.html)

In [82]:
#Install Dependencies and SkTime
!pip install tensorflow
!pip install sktime[dl]
!pip install sktime[all_extras]

  Using cached pyarrow-6.0.1-cp39-cp39-win_amd64.whl.metadata (2.9 kB)
Using cached pyarrow-6.0.1-cp39-cp39-win_amd64.whl (15.5 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.0.0
    Uninstalling pyarrow-18.0.0:
      Successfully uninstalled pyarrow-18.0.0
  Using cached pyarrow-18.0.0-cp39-cp39-win_amd64.whl.metadata (3.4 kB)
Using cached pyarrow-18.0.0-cp39-cp39-win_amd64.whl (25.1 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [83]:
#Import sktime
import sktime
#Import classifier to be used
from sktime.classification.deep_learning.cnn import CNNClassifier
#Import datasets currently available in sktime
from sktime.datasets import load_basic_motions

In [84]:
# load an example time series panel in pd-multiindex mtype
X, Y = load_basic_motions(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["accel_1", "accel_2", "accel_3", "gyro_1", "gyro_2", "gyro_3"]
X.index.names = ["trial_no", "timepoint"]

The basic motions dataset has:

- 80 individual time series instances = trials = person engaging in an activity like running, badminton, etc.
- Six variables per time series instance, dim_0 to dim_5 (renamed according to the values they represent)
- 3 accelerometer and 3 gyrometer measurements (Multi-variate dataset)
- Individual time series are observed at 100 time points (the same number for all instances)

In [85]:
# The outermost index represents the instance number
# whereas the inner index represents the index of the particular index
# within that instance.
X

accel_1    accel_2   accel_3    gyro_1    gyro_2  \
trial_no timepoint                                                       
0        0           0.079106   0.394032  0.551444  0.351565  0.023970   
         1           0.079106   0.394032  0.551444  0.351565  0.023970   
         2          -0.903497  -3.666397 -0.282844 -0.095881 -0.319605   
         3           1.116125  -0.656101  0.333118  1.624657 -0.569962   
         4           1.638200   1.405135  0.393875  1.187864 -0.271664   
...                       ...        ...       ...       ...       ...   
79       95         28.459024 -16.633770  3.631869  8.978229 -3.611533   
         96         10.260094   0.102775  1.269261 -1.645964 -3.377157   
         97          4.316471  -3.574319  2.063831 -1.717875 -1.843054   
         98          0.704446  -4.920444  2.851857 -2.982977 -0.809665   
         99         -2.074749  -6.892377  4.848379 -1.350330 -1.203844   

                      gyro_3  
trial_no timepoint            
0        0          0.633883  
         1          0.633883  
         2          0.972131  
         3          1.209171  
         4          1.739182  
...                      ...  
79       95        -1.491489  
         96         1.283746  
         97         0.484734  
         98        -0.721774  
         99        -1.776470  

[8000 rows x 6 columns]

In [86]:
# Check the labels for each X value
Y

array(['standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'running', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'walking', 'walking',
       'walking', 'walking', 'walking', 'walking', 'walking', 'walking',
       'walking', 'walking', 'badminton', 'badminton', 'badminton',
       'badminton', 'badminton', 'badminton', 'badminton', 'badminton',
       'badminton', 'badminton', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'running', 'running',
       'walking', 'walking', 'walking', 'walking', 'walking', 'walking',
       'walking', 'walking', 'walking', 'walking', 'badminton',
       'badminton', 'badminton', 'badminton', 'badminton', 'badminton',
       'badmin

In [87]:
# Check length of labels
Y.shape

(80,)

In [88]:
# Select:
# * the fourth variable (gyroscope 1)
# * of the twentieth instance (trial 20 in python)
# * values at all 100 timestamps
X.loc[20, "gyro_1"]

timepoint
0    -1.033389
1    -1.033389
2     0.066584
3     0.599259
4     0.663180
        ...   
95    0.210406
96    0.870923
97    0.870923
98    1.600687
99   -0.170456
Name: gyro_1, Length: 100, dtype: float64

In [89]:
# Select:
# * the 1st variable (accelerometer 1)
# * of the 79th instance (trial 79 in python)
# * the 98th time point
X.loc[(79, 98), "accel_1"]

0.704446

In [90]:
# Splitting the same file into training and testing set
X_train, y_train = load_basic_motions(split="train", return_type="numpy3D")
X_test, y_test = load_basic_motions(split="test", return_type="numpy3D")

In [91]:
# Sanity check to find training features and truth labels
print(X_train.shape)
print(y_train.shape)

(40, 6, 100)
(40,)


In [92]:
#Import CNN classifier
from sktime.classification.deep_learning.cnn import CNNClassifier

In [93]:
#Define the CNN classifier
cnn = CNNClassifier(n_epochs=20,batch_size=4)
cnn.fit(X_train, y_train)

CNNClassifier(batch_size=4, n_epochs=20)

In [94]:
# nested parameter interface via get_params, set_params
cnn.get_params()

{'activation': 'softmax',
 'avg_pool_size': 3,
 'batch_size': 4,
 'callbacks': None,
 'filter_sizes': None,
 'kernel_size': 7,
 'loss': 'categorical_crossentropy',
 'metrics': None,
 'n_conv_layers': 2,
 'n_epochs': 20,
 'optimizer': None,
 'padding': 'auto',
 'random_state': None,
 'use_bias': True,
 'verbose': False}

In [95]:
# We inspect fitted parameters if we like
cnn.get_fitted_params()

{'classes': array(['badminton', 'running', 'standing', 'walking'], dtype='<U9'),
 'fit_time': 2235,
 'model': <Functional name=functional_7, built=True>,
 'n_classes': 4,
 'optimizer': <keras.src.optimizers.adam.Adam at 0x21c2214ceb0>}

In [96]:
#Store the predictions from CNN over testing set
y_pred = cnn.predict(X_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [97]:
# Evaluate the results
from sklearn.metrics import classification_report

In [98]:
#Print evaluation metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   badminton       1.00      0.80      0.89        10
     running       1.00      1.00      1.00        10
    standing       0.91      1.00      0.95        10
     walking       0.82      0.90      0.86        10

    accuracy                           0.93        40
   macro avg       0.93      0.92      0.92        40
weighted avg       0.93      0.93      0.92        40



Try the same with Multi-Layered Perceptron..
API Reference: https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.classification.deep_learning.MLPClassifier.html